In [1]:
import numpy as np

# Utility functions to initialize the problem
from odp.Grid import Grid
from odp.Shapes import *

# Specify the  file that includes dynamic systems
from odp.dynamics import DubinsCapture, Plane2D, Plane1D, DubinsCar4D, PursuitEvasion

# Plot options
from odp.Plots import PlotOptions
from odp.Plots import plot_isosurface, plot_valuefunction

# Solver core
from odp.solver import HJSolver, computeSpatDerivArray

import math
import os

""" USER INTERFACES
- Define grid
- Generate initial values for grid using shape functions
- Time length for computations
- System dynamics for computation
- Initialize plotting option
- Call HJSolver function

Note: If run on the server, please save the result and use the plot function on your local machine
"""

if os.path.exists("plots") == False:
    os.mkdir("plots")


In [ ]:
##################################################### 4D EXAMPLE #####################################################
# STEP 1: Define grid
grid_min = np.array([-4.0, -4.0, -4.0, -math.pi])
grid_max = np.array([4.0, 4.0, 4.0, math.pi])
dims = 4
N = np.array([80, 80, 80, 80])
pd = [3]
g = Grid(grid_min, grid_max, dims, N, pd)

# STEP 2: Generate initial values for grid using shape functions
center = np.zeros(dims)
radius = 2.0
ignore_dims = [3]
Initial_value_f = CylinderShape(g, ignore_dims, center, radius)

# STEP 3: Time length for computations
Lookback_length = 0.5
t_step = 0.05

small_number = 1e-5
tau = np.arange(start=0, stop=Lookback_length + small_number, step=t_step)

# STEP 4: System dynamics for computation
sys4D = DubinsCar4D(uMode="max", dMode="min")

# STEP 5: Initialize plotting option
po = PlotOptions(
    do_plot=True,
    plot_type="set",
    plotDims=[0, 1, 3],
    slicesCut=[50],
    colorscale="Bluered",
    save_fig=False,
    filename="plots/4D_0_sublevel_set",
    interactive_html=True,
)

# STEP 6: Call HJSolver function
compMethod = {"TargetSetMode": "None"}
result_3 = HJSolver(
    sys4D, g, Initial_value_f, tau, compMethod, po, saveAllTimeSteps=True
)


In [ ]:
##################################################### 3D EXAMPLE #####################################################
# STEP 1: Define grid
grid_min = np.array([-10.0, -10.0, -math.pi])
grid_max = np.array([10.0, 10.0, math.pi])
dims = 3
N = np.array([150, 150, 150])
pd = [2]
g = Grid(grid_min, grid_max, dims, N, pd)

# STEP 2: Generate initial values for grid using shape functions
center = np.zeros(dims)
radius = 1.0
ignore_dims = [2]
Initial_value_f = CylinderShape(g, ignore_dims, center, radius)

# STEP 3: Time length for computations
Lookback_length = 1.0
t_step = 0.05

small_number = 1e-5
tau = np.arange(start=0, stop=Lookback_length + small_number, step=t_step)

# STEP 4: System dynamics for computation
sys = DubinsCapture(uMode="max", dMode="min")

# STEP 5: Initialize plotting option
po1 = PlotOptions(do_plot=True, 
        plot_type="set", 
        plotDims=[0, 1, 2], 
        slicesCut=[50], 
        colorscale="Bluered", 
        save_fig=False, 
        filename="plots/3D_0_sublevel_set", 
        interactive_html=True)


# po = PlotOptions(
#     do_plot=True,
#     plot_type="set",
#     plotDims=[0, 1, 3],
#     slicesCut=[50],
#     colorscale="Bluered",
#     save_fig=False,
#     filename="plots/4D_0_sublevel_set",
#     interactive_html=True,
# )


# STEP 6: Call HJSolver function
compMethod = {"TargetSetMode": "None"}
result_3 = HJSolver(
    sys, g, Initial_value_f, tau, compMethod, po1, saveAllTimeSteps=True
)

"""
Test downsample function
"""
# print(result_3[:,:,:,1].shape)
# print(g.dims)
# g_out, data_out = downsample(g, result_3, [2,2,2])

# print(data_out.shape)


# While file needs to be saved locally, set save_fig=True and filename, recommend to set interactive_html=True for better visualization

# po2 = PlotOptions(
#     do_plot=False,
#     plot_type="set",
#     plotDims=[0, 1, 2],
#     slicesCut=[1],
#     colorscale="Bluered",
#     save_fig=True,
#     filename="plots/3D_0_sublevel_set",
#     interactive_html=True,
# )
# 
# # STEP 6: Call Plotting function
# plot_isosurface(g, result_3, po2)


last_time_step_result = result_3[..., 0]
# Compute spatial derivatives at every state
x_derivative = computeSpatDerivArray(
    g, last_time_step_result, deriv_dim=1, accuracy="low"
)
y_derivative = computeSpatDerivArray(
    g, last_time_step_result, deriv_dim=2, accuracy="low"
)
T_derivative = computeSpatDerivArray(
    g, last_time_step_result, deriv_dim=3, accuracy="low"
)

# Let's compute optimal control at some random idices
spat_deriv_vector = (
    x_derivative[10, 20, 30],
    y_derivative[10, 20, 30],
    T_derivative[10, 20, 30],
)
state_vector = (g.grid_points[0][10], g.grid_points[1][20], g.grid_points[2][30])

# Compute the optimal control
opt_ctrl = sys.optCtrl_inPython(state_vector, spat_deriv_vector)

In [ ]:
print(g.grid_points)

In [ ]:
##################################################### 2D EXAMPLE #####################################################
# STEP 1: Define grid
grid_min = np.array([-4.0, -4.0])
grid_max = np.array([4.0, 4.0])
dims = 2
N = np.array([150, 150])
g_2 = Grid(grid_min, grid_max, dims, N)

# STEP 2: Generate initial values for grid using shape functions
target_min = np.array([-1.0, -1.0])
target_max = np.array([1.0, 1.0])
Initial_value_f = ShapeRectangle(g_2, target_min, target_max)

# STEP 3: Time length for computations
Lookback_length = 2.0
t_step = 0.05

small_number = 1e-5
tau = np.arange(start=0, stop=Lookback_length + small_number, step=t_step)

# STEP 4: System dynamics for computation
sys = Plane2D()

# STEP 5: Initialize plotting option
po1 = PlotOptions(do_plot=True, plot_type="value", plotDims=[0, 1])

# STEP 6: Call HJSolver function
compMethod = {"TargetSetMode": "None"}
result_2 = HJSolver(
    sys, g_2, Initial_value_f, tau, compMethod, po1, saveAllTimeSteps=True
)

# Visualization of animated 2D value function
po2 = PlotOptions(
    do_plot=False,
    plot_type="value",
    plotDims=[0, 1],
    slicesCut=[50],
    colorscale="Hot",
    save_fig=True,
    filename="plots/2D_0_valuefunction",
    interactive_html=True,
)
plot_valuefunction(g, result_3, po2)

# Visualization of animated 2D 0 sublevel set
po3 = PlotOptions(
    do_plot=False,
    plot_type="set",
    plotDims=[0, 1],
    slicesCut=[50],
    colorscale="Bluered",
    save_fig=True,
    filename="plots/2D_0_sublevel_set",
    interactive_html=True,
)
plot_isosurface(g, result_3, po3)


In [ ]:
# ##################################################### 1D EXAMPLE #####################################################
# STEP 1: Define grid
grid_min_1 = np.array([-4.0])
grid_max_1 = np.array([4.0])
dims_1 = 1
N_1 = np.array([150])
g_1 = Grid(grid_min_1, grid_max_1, dims_1, N_1)

# STEP 2: Generate initial values for grid using shape functions
target_min_1 = np.array([-1.0])
target_max_1 = np.array([1.0])
Initial_value_f_1 = ShapeRectangle(g_1, target_min_1, target_max_1)

# print(Initial_value_f_1.shape)

# STEP 3: Time length for computations
Lookback_length = 2.0
t_step = 0.05

small_number = 1e-5
tau = np.arange(start=0, stop=Lookback_length + small_number, step=t_step)

# STEP 4: System dynamics for computation
sys_1 = Plane1D()

# STEP 5: Initialize plotting option
po1 = PlotOptions(do_plot=False, plot_type="value", plotDims=[0], slicesCut=[])

# STEP 6: Call HJSolver function
compMethod = {"TargetSetMode": "None"}

result_1 = HJSolver(
    sys_1, g_1, Initial_value_f_1, tau, compMethod, po1, saveAllTimeSteps=True
)

po2 = PlotOptions(
    do_plot=False,
    plot_type="value",
    plotDims=[0],
    slicesCut=[50, 50],
    save_fig=True,
    filename="plots/1D_0_valuefunction.png",
    interactive_html=False,
)
plot_valuefunction(g, result_3, po2)


In [2]:
##################################################### TWINPMPE EXAMPLE #####################################################
# STEP 1: Define grid
grid_min = np.array([-20.0, -20.0, -4.5, -4.5])
grid_max = np.array([20.0, 20.0, 4.5, 4.5])
dims = 4
N = np.array([80, 80, 80, 80])
pd = []
g = Grid(grid_min, grid_max, dims, N, pd)

# STEP 2: Generate initial values for grid using shape functions
radius = 2.0
ignore_dims = [2,3]
Initial_value_f = CylinderShape(g, ignore_dims, np.zeros(dims), radius)
# STEP 3: Time length for computations
Lookback_length = 0.5
t_step = 0.05

small_number = 1e-5
tau = np.arange(start=0, stop=Lookback_length + small_number, step=t_step)

# STEP 4: System dynamics for computation
sys = PursuitEvasion(uMode="max", dMode="min")

# STEP 5: Initialize plotting option
po = PlotOptions(
    do_plot=True,
    plot_type="set",
    plotDims=[0, 1],
    slicesCut=[],
    colorscale="Bluered",
    save_fig=False,
    filename="plots/TwinPMPE_0_sublevel_set",
    interactive_html=True,
)

# STEP 6: Call HJSolver function
compMethod = {"TargetSetMode": "minVWithV0"}
result = HJSolver(sys, g, Initial_value_f, tau, compMethod, po, saveAllTimeSteps=True)



Welcome to optimized_dp 

Initializing

No obstacles set !
Optimizing

(80, 80, 80, 80, 11)
Started running

[0.01512685 0.05      ]
Computational time to integrate (s): 1.72250
[0.03025369 0.05      ]
Computational time to integrate (s): 1.45720
[0.04538054 0.05      ]
Computational time to integrate (s): 1.17911
[0.05 0.05]
Computational time to integrate (s): 1.32732
[0.06512685 0.1       ]
Computational time to integrate (s): 1.21924
[0.08025369 0.1       ]
Computational time to integrate (s): 1.17696
[0.09538054 0.1       ]
Computational time to integrate (s): 1.67003
[0.1 0.1]
Computational time to integrate (s): 1.46875
[0.11512685 0.15      ]
Computational time to integrate (s): 1.18130
[0.13025369 0.15      ]
Computational time to integrate (s): 1.24461
[0.14538054 0.15      ]
Computational time to integrate (s): 1.16083
[0.15 0.15]
Computational time to integrate (s): 1.19555
[0.16512685 0.2       ]
Computational time to integrate (s): 1.23478
[0.18025369 0.2       ]
Computat

IndexError: list index out of range